# JHU CSSE GIS Dashboard
* https://www.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6
* https://coronavirus.jhu.edu/map.html
* https://www.arcgis.com/home/item.html?id=bbb2e4f589ba40d692fab712ae37b9ac#data

In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

In [2]:
data = requests.get(
    'https://services1.arcgis.com/0MSEUqKaxRlEPj5g/arcgis/rest/services/Coronavirus_2019_nCoV_Cases/FeatureServer/2/query?f=json&where=1%3D1&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&orderByFields=OBJECTID%20ASC&resultOffset=0&resultRecordCount=200&cacheHint=true&quantizationParameters=%7B%22mode%22%3A%22edit%22%7D').json()

In [3]:
country_list = []
for country in data['features']:
    country_dic = {}
    country_dic['OBJECTID'] = country['attributes']['OBJECTID']
    country_dic['Country_Region'] = country['attributes']['Country_Region']
    country_dic['Last_Update'] = country['attributes']['Last_Update']
    country_dic['Lat'] = country['attributes']['Lat']
    country_dic['Long_'] = country['attributes']['Long_']
    country_dic['Confirmed'] = country['attributes']['Confirmed']
    country_dic['Deaths'] = country['attributes']['Deaths']
    country_dic['Recovered'] = country['attributes']['Recovered']
    country_list.append(country_dic)

In [4]:
df = pd.DataFrame(country_list)

In [5]:
df.head()

,OBJECTID,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered
0,1,Australia,1585744394000,-25.0000,133.0000,4862,20,422
1,2,Austria,1585744036000,47.5162,14.5501,10444,146,1436
2,3,Canada,1585744378000,60.0010,-95.0010,8591,102,1592
3,4,China,1585739259000,30.5928,114.3055,82361,3316,76401
4,5,Denmark,1585744036000,56.0000,10.0000,3275,104,78


In [6]:
df.Last_Update = pd.to_datetime(
    df.Last_Update/1000,unit='s').dt.tz_localize('utc').dt.tz_convert('Asia/Tokyo').dt.strftime('%Y-%m-%d %H:%M')

In [7]:
df.head()

,OBJECTID,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered
0,1,Australia,2020-04-01 21:33,-25.0000,133.0000,4862,20,422
1,2,Austria,2020-04-01 21:27,47.5162,14.5501,10444,146,1436
2,3,Canada,2020-04-01 21:32,60.0010,-95.0010,8591,102,1592
3,4,China,2020-04-01 20:07,30.5928,114.3055,82361,3316,76401
4,5,Denmark,2020-04-01 21:27,56.0000,10.0000,3275,104,78


In [8]:
df['Active'] = df.Confirmed - df.Deaths - df.Recovered

In [9]:
df.to_csv('data/latest_global_'+datetime.now().strftime('%Y%m%d_%I%M%p')+'.csv',index=False,encoding='utf-8-sig')

In [10]:
countries = pd.read_csv('countries.csv')

In [11]:
countries.head()

,Country,Japanese,Code,Region
0,Afghanistan,アフガニスタン,AF,Asia
1,Albania,アルバニア,AL,Europe
2,Algeria,アルジェリア,DZ,Africa
3,Andorra,アンドラ,AD,Europe
4,Angola,アンゴラ,AO,Africa


In [12]:
df.Country_Region = df.Country_Region.map(countries.set_index('Country').Japanese.to_dict())

In [13]:
df.head()

,OBJECTID,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active
0,1,オーストラリア,2020-04-01 21:33,-25.0000,133.0000,4862,20,422,4420
1,2,オーストリア,2020-04-01 21:27,47.5162,14.5501,10444,146,1436,8862
2,3,カナダ,2020-04-01 21:32,60.0010,-95.0010,8591,102,1592,6897
3,4,中国,2020-04-01 20:07,30.5928,114.3055,82361,3316,76401,2644
4,5,デンマーク,2020-04-01 21:27,56.0000,10.0000,3275,104,78,3093


In [14]:
df.columns = ['ID','国名','更新時間（日本時間）','緯度','経度','累積感染者数','累積死者数','累積回復者数','現行感染者数']

In [15]:
df.head()

,ID,国名,更新時間（日本時間）,緯度,経度,累積感染者数,累積死者数,累積回復者数,現行感染者数
0,1,オーストラリア,2020-04-01 21:33,-25.0000,133.0000,4862,20,422,4420
1,2,オーストリア,2020-04-01 21:27,47.5162,14.5501,10444,146,1436,8862
2,3,カナダ,2020-04-01 21:32,60.0010,-95.0010,8591,102,1592,6897
3,4,中国,2020-04-01 20:07,30.5928,114.3055,82361,3316,76401,2644
4,5,デンマーク,2020-04-01 21:27,56.0000,10.0000,3275,104,78,3093


In [16]:
summary = df[['累積感染者数','累積死者数','累積回復者数','現行感染者数']].sum().reset_index()

In [17]:
summary.columns = ['項目','合計']

In [18]:
summary = summary.set_index('項目').T

In [19]:
summary.index = [datetime.today().strftime('%Y-%m-%d')]

In [20]:
summary

項目,累積感染者数,累積死者数,累積回復者数,現行感染者数
2020-04-01,877422,43537,185241,648644


In [21]:
total = pd.read_excel('covid19_'+datetime.today().strftime('%Y%m%d')+'.xlsx',sheet_name='一覧',index_col=0
                     ).append(summary,sort=True).sort_index(ascending=False)

In [22]:
total.head()

,新規回復者数,新規感染者数,新規死者数,現行感染者増減数,現行感染者数,累積回復者数,累積感染者数,累積死者数
2020-04-01,NaN,NaN,NaN,NaN,648644,185241,877422,43537
2020-03-31,13468.0,75122.0,4525.0,57129.0,637346,178034,857487,42107
2020-03-30,15484.0,62248.0,3657.0,43107.0,580217,164566,782365,37582
2020-03-29,9667.0,59411.0,3273.0,46471.0,537110,149082,720117,33925
2020-03-28,8500.0,67415.0,3454.0,55461.0,490639,139415,660706,30652


In [23]:
total.iloc[0,0] = total.iloc[0,5] - total.iloc[1,5]

In [24]:
total.iloc[0,1] = total.iloc[0,6] - total.iloc[1,6]

In [25]:
total.iloc[0,2] = total.iloc[0,7] - total.iloc[1,7]

In [26]:
total.iloc[0,3] = total.iloc[0,4] - total.iloc[1,4]

In [27]:
total = total.astype(int)

In [28]:
total.head()

,新規回復者数,新規感染者数,新規死者数,現行感染者増減数,現行感染者数,累積回復者数,累積感染者数,累積死者数
2020-04-01,7207,19935,1430,11298,648644,185241,877422,43537
2020-03-31,13468,75122,4525,57129,637346,178034,857487,42107
2020-03-30,15484,62248,3657,43107,580217,164566,782365,37582
2020-03-29,9667,59411,3273,46471,537110,149082,720117,33925
2020-03-28,8500,67415,3454,55461,490639,139415,660706,30652


In [29]:
with pd.ExcelWriter('covid19_latest_'+datetime.now().strftime('%Y%m%d_%I%M%p')+'.xlsx') as writer:  
    df.set_index('国名').sort_values('累積感染者数',ascending=False).drop(['ID'],axis=1).to_excel(writer,sheet_name='最新')
    total.to_excel(writer,sheet_name='合計')